In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [8]:
cascade_path = "./haarcascade_frontalface_default.xml"
face_detection = cv2.CascadeClassifier(cascade_path)

In [9]:

# Carregar o modelo de reconhecimento de expressões faciais
model_path = "./modelo_01_expressoes.keras"
model = load_model(model_path, compile=False)


In [10]:
# Definir as expressões
expressoes = ["raiva", "nojo", "medo", "feliz", "triste", "surpreso", "neutro"]

In [11]:
# Iniciar a captura de vídeo
cap = cv2.VideoCapture(0)  # 0 é o índice da câmera padrão

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Converter para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar faces na imagem
    faces = face_detection.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))
    
    for (x, y, w, h) in faces:
        # Extrair a região de interesse (ROI)
        roi = gray[y:y + h, x:x + w]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Fazer a previsão da expressão facial
        preds = model.predict(roi)[0]
        label = expressoes[np.argmax(preds)]
        
        # Desenhar um retângulo ao redor do rosto e adicionar o rótulo
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Mostrar o frame com as detecções
    cv2.imshow("Emoção Facial", frame)
    
    # Parar a captura se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [12]:
cap.release()
cv2.destroyAllWindows()